# Day 6: Variational Inference for Scalable Bayesian Learning

## Week 20: Bayesian Methods | Quant ML Learning Path

---

### 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why Variational Inference (VI) is essential for scalable Bayesian learning
2. **Master** the Evidence Lower Bound (ELBO) derivation and optimization
3. **Implement** Mean-Field Variational Inference from scratch
4. **Apply** Coordinate Ascent VI (CAVI) and Stochastic VI (SVI) algorithms
5. **Use** Automatic Differentiation VI (ADVI) with PyMC
6. **Compare** VI vs MCMC for practical applications in finance

---

### 📚 Table of Contents

1. [Introduction to Variational Inference](#1-introduction)
2. [Understanding KL Divergence](#2-kl-divergence)
3. [Evidence Lower Bound (ELBO) Derivation](#3-elbo)
4. [Mean-Field Variational Inference](#4-mean-field)
5. [VI for Gaussian Mixture Models](#5-gmm)
6. [Coordinate Ascent Variational Inference (CAVI)](#6-cavi)
7. [Stochastic Variational Inference (SVI)](#7-svi)
8. [Automatic Differentiation VI (ADVI)](#8-advi)
9. [Variational Inference with PyMC](#9-pymc)
10. [Comparing VI to MCMC](#10-comparison)
11. [Financial Application: Scalable Factor Models](#11-finance)
12. [Exercises & Interview Questions](#12-exercises)

---

### 🔑 Key Insight

> **"Variational Inference transforms intractable Bayesian inference into an optimization problem, enabling scalability to millions of data points while MCMC struggles with thousands."**

In quantitative finance, where we deal with massive datasets of tick data, order book snapshots, and alternative data streams, VI provides the scalability necessary for production-grade Bayesian models.

## 1. Import Required Libraries <a name="1-introduction"></a>

In [ ]:
# Core Scientific Computing
import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import digamma, gammaln, logsumexp
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# Probabilistic Programming
import pymc as pm
import arviz as az

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs

# Deep Learning for VI (optional - for ADVI from scratch)
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.distributions import Normal, MultivariateNormal, kl_divergence
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False
    print("PyTorch not available - some examples will be skipped")

# Styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14

# Random seeds for reproducibility
np.random.seed(42)
if TORCH_AVAILABLE:
    torch.manual_seed(42)

print("✅ All libraries loaded successfully!")
print(f"PyMC version: {pm.__version__}")
print(f"PyTorch available: {TORCH_AVAILABLE}")

### Why Variational Inference?

**The Scalability Problem with MCMC:**

Traditional Bayesian inference using MCMC (Markov Chain Monte Carlo) faces significant challenges:
- **Computational Cost**: $O(N)$ per iteration, where $N$ is the dataset size
- **Convergence**: Requires many iterations to explore the posterior
- **Parallelization**: Sequential nature limits GPU utilization
- **Large Data**: Impractical for datasets with millions of observations

**Variational Inference Solution:**

VI reframes Bayesian inference as an **optimization problem**:
- Instead of sampling from $p(\theta|D)$, we find the closest approximation $q(\theta)$
- Leverages gradient descent and automatic differentiation
- Enables **mini-batch training** for massive datasets
- Naturally parallelizable on GPUs

$$\text{MCMC: Sample from } p(\theta|D) \quad \rightarrow \quad \text{VI: Find } q^*(\theta) = \arg\min_q D_{KL}(q(\theta) \| p(\theta|D))$$

| Aspect | MCMC | Variational Inference |
|--------|------|----------------------|
| Output | Samples from posterior | Parameters of approximating distribution |
| Convergence | Asymptotically exact | Approximate (but fast) |
| Scalability | $O(N \times T)$ | $O(M \times T)$ with minibatches |
| GPU-friendly | Limited | Excellent |
| Uncertainty | Full posterior | Approximation (often underestimates) |

---

## 2. Understanding KL Divergence <a name="2-kl-divergence"></a>

The **Kullback-Leibler (KL) Divergence** is the foundation of variational inference. It measures how different one probability distribution is from another.

### Mathematical Definition

For two distributions $p$ and $q$:

$$D_{KL}(q \| p) = \mathbb{E}_q\left[\log \frac{q(z)}{p(z)}\right] = \int q(z) \log \frac{q(z)}{p(z)} dz$$

**Key Properties:**
1. **Non-negative**: $D_{KL}(q \| p) \geq 0$
2. **Zero iff equal**: $D_{KL}(q \| p) = 0 \Leftrightarrow q = p$
3. **Asymmetric**: $D_{KL}(q \| p) \neq D_{KL}(p \| q)$

### Forward vs Reverse KL

The asymmetry is crucial for understanding VI behavior:

- **Forward KL** $D_{KL}(p \| q)$: "Mode-covering" - $q$ covers all modes of $p$
- **Reverse KL** $D_{KL}(q \| p)$: "Mode-seeking" - $q$ concentrates on a single mode

VI minimizes **Reverse KL**, leading to posterior approximations that may miss modes but are computationally tractable.

In [ ]:
def kl_divergence_gaussians(mu_q, sigma_q, mu_p, sigma_p):
    """
    Compute KL divergence between two univariate Gaussians.
    
    D_KL(q || p) = log(σ_p/σ_q) + (σ_q² + (μ_q - μ_p)²) / (2σ_p²) - 1/2
    """
    return (np.log(sigma_p / sigma_q) + 
            (sigma_q**2 + (mu_q - mu_p)**2) / (2 * sigma_p**2) - 0.5)


def kl_divergence_numerical(q_samples, p_pdf, q_pdf):
    """
    Numerical approximation of KL divergence using samples.
    D_KL(q || p) ≈ (1/N) Σ [log q(x_i) - log p(x_i)]
    """
    log_q = np.log(q_pdf(q_samples) + 1e-10)
    log_p = np.log(p_pdf(q_samples) + 1e-10)
    return np.mean(log_q - log_p)


# Demonstrate KL divergence asymmetry
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Target distribution: bimodal mixture
x = np.linspace(-5, 5, 1000)
p_bimodal = 0.5 * stats.norm.pdf(x, -2, 0.7) + 0.5 * stats.norm.pdf(x, 2, 0.7)

# Different approximations
approximations = [
    ("Mode-covering (Wide Gaussian)", 0, 2.5),
    ("Mode-seeking (Left mode)", -2, 0.7),
    ("Mode-seeking (Right mode)", 2, 0.7)
]

for ax, (title, mu, sigma) in zip(axes, approximations):
    q = stats.norm.pdf(x, mu, sigma)
    
    ax.fill_between(x, p_bimodal, alpha=0.3, color='blue', label='True posterior p(z|x)')
    ax.fill_between(x, q, alpha=0.3, color='red', label=f'Approximation q(z)')
    ax.plot(x, p_bimodal, 'b-', lw=2)
    ax.plot(x, q, 'r--', lw=2)
    
    # Compute KL divergences numerically
    samples = np.random.normal(mu, sigma, 10000)
    q_pdf = lambda z: stats.norm.pdf(z, mu, sigma)
    p_pdf = lambda z: 0.5 * stats.norm.pdf(z, -2, 0.7) + 0.5 * stats.norm.pdf(z, 2, 0.7)
    
    kl_q_p = kl_divergence_numerical(samples, p_pdf, q_pdf)
    
    ax.set_title(f'{title}\nKL(q||p) ≈ {kl_q_p:.3f}')
    ax.set_xlabel('z')
    ax.set_ylabel('Density')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(-5, 5)

plt.suptitle('KL Divergence: Mode-Covering vs Mode-Seeking Behavior', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("💡 Key Insight:")
print("   - Forward KL (mode-covering): Places mass everywhere p has mass → wide approximation")
print("   - Reverse KL (mode-seeking): Concentrates on high-density regions → may miss modes")
print("   - VI uses reverse KL, explaining why it can underestimate posterior uncertainty")

In [ ]:
# Interactive exploration of KL divergence between Gaussians
def explore_kl_divergence():
    """
    Visualize how KL divergence changes as q deviates from p.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Fixed target p ~ N(0, 1)
    mu_p, sigma_p = 0, 1
    x = np.linspace(-5, 5, 500)
    p = stats.norm.pdf(x, mu_p, sigma_p)
    
    # Vary mean of q
    ax = axes[0]
    mu_range = np.linspace(-3, 3, 100)
    kl_values = [kl_divergence_gaussians(mu_q, 1, mu_p, sigma_p) for mu_q in mu_range]
    
    ax.plot(mu_range, kl_values, 'b-', lw=2)
    ax.axvline(0, color='red', linestyle='--', label='Optimal μ_q = μ_p')
    ax.set_xlabel('μ_q (mean of q)')
    ax.set_ylabel('KL(q || p)')
    ax.set_title('KL Divergence vs Mean Shift\n(σ_q = σ_p = 1)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Vary variance of q
    ax = axes[1]
    sigma_range = np.linspace(0.1, 3, 100)
    kl_values = [kl_divergence_gaussians(0, sigma_q, mu_p, sigma_p) for sigma_q in sigma_range]
    
    ax.plot(sigma_range, kl_values, 'b-', lw=2)
    ax.axvline(1, color='red', linestyle='--', label='Optimal σ_q = σ_p')
    ax.set_xlabel('σ_q (std of q)')
    ax.set_ylabel('KL(q || p)')
    ax.set_title('KL Divergence vs Variance Mismatch\n(μ_q = μ_p = 0)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

explore_kl_divergence()

print("\n📊 Observations:")
print("   - KL divergence is minimized when q matches p exactly")
print("   - Mean shifts cause quadratic increase in KL")
print("   - Both underestimating and overestimating variance increases KL")
print("   - But underestimating variance (narrow q) is more heavily penalized")

---

## 3. Evidence Lower Bound (ELBO) Derivation <a name="3-elbo"></a>

The **Evidence Lower Bound (ELBO)** is the cornerstone of variational inference. We derive it by showing that maximizing ELBO is equivalent to minimizing KL divergence.

### The Problem

We want to compute the posterior $p(z|x)$ but it's intractable because of the normalizing constant:

$$p(z|x) = \frac{p(x|z)p(z)}{p(x)} = \frac{p(x|z)p(z)}{\int p(x|z)p(z)dz}$$

The marginal likelihood $p(x) = \int p(x|z)p(z)dz$ is often intractable.

### ELBO Derivation

Starting from the log marginal likelihood (log evidence):

$$\log p(x) = \log \int p(x, z) dz$$

For any distribution $q(z)$:

$$\log p(x) = \log \int \frac{p(x, z)}{q(z)} q(z) dz$$

By Jensen's inequality ($\log \mathbb{E}[X] \geq \mathbb{E}[\log X]$):

$$\log p(x) \geq \int q(z) \log \frac{p(x, z)}{q(z)} dz = \mathcal{L}(q)$$

This gives us the **ELBO**:

$$\boxed{\mathcal{L}(q) = \mathbb{E}_q[\log p(x, z)] - \mathbb{E}_q[\log q(z)] = \mathbb{E}_q[\log p(x|z)] - D_{KL}(q(z) \| p(z))}$$

### Two Equivalent Forms of ELBO

**Form 1 (Reconstruction + KL):**
$$\mathcal{L}(q) = \underbrace{\mathbb{E}_q[\log p(x|z)]}_{\text{Reconstruction}} - \underbrace{D_{KL}(q(z) \| p(z))}_{\text{Regularization}}$$

**Form 2 (Joint - Entropy):**
$$\mathcal{L}(q) = \underbrace{\mathbb{E}_q[\log p(x, z)]}_{\text{Expected log joint}} + \underbrace{\mathcal{H}[q(z)]}_{\text{Entropy of q}}$$

### Connection to KL Divergence

$$\log p(x) = \mathcal{L}(q) + D_{KL}(q(z) \| p(z|x))$$

Since $D_{KL} \geq 0$, maximizing $\mathcal{L}(q)$ is equivalent to minimizing $D_{KL}(q \| p(z|x))$!

In [ ]:
class ELBOVisualization:
    """
    Visualize the ELBO and its relationship to the log evidence and KL divergence.
    """
    
    def __init__(self, true_mu=2.0, true_sigma=1.0, prior_mu=0.0, prior_sigma=2.0, n_obs=50):
        """
        Simple Bayesian inference problem: estimate mean of Gaussian with known variance.
        
        Likelihood: x_i | μ ~ N(μ, σ²_known)
        Prior: μ ~ N(μ_0, σ²_0)
        Posterior: μ | x ~ N(μ_n, σ²_n) [conjugate, analytically tractable]
        """
        self.prior_mu = prior_mu
        self.prior_sigma = prior_sigma
        self.likelihood_sigma = 1.0  # Known observation noise
        
        # Generate data
        np.random.seed(42)
        self.data = np.random.normal(true_mu, self.likelihood_sigma, n_obs)
        self.n = len(self.data)
        self.data_mean = np.mean(self.data)
        
        # Analytical posterior (conjugate)
        self.posterior_precision = 1/prior_sigma**2 + self.n/self.likelihood_sigma**2
        self.posterior_sigma = 1/np.sqrt(self.posterior_precision)
        self.posterior_mu = (prior_mu/prior_sigma**2 + 
                            self.n * self.data_mean/self.likelihood_sigma**2) / self.posterior_precision
        
        # Log evidence (analytically computable for this simple case)
        self._compute_log_evidence()
    
    def _compute_log_evidence(self):
        """Compute log p(x) analytically for Gaussian-Gaussian model."""
        # p(x) = ∫ p(x|μ)p(μ) dμ
        # For Gaussian conjugate: p(x) ~ N(μ_0, σ²_0 + σ²_likelihood/n)
        predictive_var = self.prior_sigma**2 + self.likelihood_sigma**2/self.n
        self.log_evidence = stats.norm.logpdf(self.data_mean, self.prior_mu, np.sqrt(predictive_var))
        # Adjust for full data
        self.log_evidence = np.sum(stats.norm.logpdf(
            self.data, self.data_mean, self.likelihood_sigma
        )) + self.log_evidence
    
    def compute_elbo(self, q_mu, q_sigma):
        """
        Compute ELBO for variational distribution q(μ) = N(q_mu, q_sigma²).
        
        ELBO = E_q[log p(x|μ)] + E_q[log p(μ)] - E_q[log q(μ)]
        """
        # E_q[log p(x|μ)] - expected log likelihood
        # = -n/2 log(2π) - n/2 log(σ²) - 1/(2σ²) E_q[Σ(x_i - μ)²]
        # = -n/2 log(2πσ²) - 1/(2σ²) [Σ(x_i - E[μ])² + n*Var[μ]]
        
        sum_sq = np.sum((self.data - q_mu)**2)
        expected_log_lik = (-self.n/2 * np.log(2*np.pi*self.likelihood_sigma**2) 
                          - 1/(2*self.likelihood_sigma**2) * (sum_sq + self.n * q_sigma**2))
        
        # E_q[log p(μ)] - expected log prior
        # = -1/2 log(2π) - log(σ_0) - 1/(2σ_0²) E_q[(μ - μ_0)²]
        expected_log_prior = (-0.5 * np.log(2*np.pi*self.prior_sigma**2)
                             - 1/(2*self.prior_sigma**2) * ((q_mu - self.prior_mu)**2 + q_sigma**2))
        
        # -E_q[log q(μ)] = H[q] (entropy)
        entropy = 0.5 * np.log(2*np.pi*np.e*q_sigma**2)
        
        elbo = expected_log_lik + expected_log_prior + entropy
        return elbo
    
    def compute_kl_to_posterior(self, q_mu, q_sigma):
        """Compute KL(q || posterior)."""
        return kl_divergence_gaussians(q_mu, q_sigma, self.posterior_mu, self.posterior_sigma)
    
    def visualize(self):
        """Visualize ELBO landscape and its relationship to KL divergence."""
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Plot 1: Distributions
        ax = axes[0, 0]
        x = np.linspace(-2, 5, 500)
        ax.plot(x, stats.norm.pdf(x, self.prior_mu, self.prior_sigma), 
                'g--', lw=2, label=f'Prior: N({self.prior_mu}, {self.prior_sigma}²)')
        ax.plot(x, stats.norm.pdf(x, self.posterior_mu, self.posterior_sigma), 
                'b-', lw=2, label=f'Posterior: N({self.posterior_mu:.2f}, {self.posterior_sigma:.3f}²)')
        ax.axvline(self.data_mean, color='red', linestyle=':', label=f'Data mean: {self.data_mean:.2f}')
        ax.set_xlabel('μ')
        ax.set_ylabel('Density')
        ax.set_title('Prior and True Posterior')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Plot 2: ELBO surface
        ax = axes[0, 1]
        mu_range = np.linspace(0, 4, 100)
        sigma_range = np.linspace(0.05, 1.5, 100)
        MU, SIGMA = np.meshgrid(mu_range, sigma_range)
        ELBO = np.zeros_like(MU)
        
        for i in range(len(sigma_range)):
            for j in range(len(mu_range)):
                ELBO[i, j] = self.compute_elbo(MU[i, j], SIGMA[i, j])
        
        contour = ax.contourf(MU, SIGMA, ELBO, levels=50, cmap='viridis')
        ax.scatter(self.posterior_mu, self.posterior_sigma, c='red', s=100, 
                   marker='*', label=f'Optimal: ({self.posterior_mu:.2f}, {self.posterior_sigma:.3f})')
        ax.set_xlabel('q_μ')
        ax.set_ylabel('q_σ')
        ax.set_title('ELBO Surface')
        ax.legend()
        plt.colorbar(contour, ax=ax, label='ELBO')
        
        # Plot 3: ELBO vs KL relationship
        ax = axes[1, 0]
        elbo_values = []
        kl_values = []
        
        for sigma in np.linspace(0.1, 1.0, 20):
            for mu in np.linspace(0.5, 3.5, 20):
                elbo_values.append(self.compute_elbo(mu, sigma))
                kl_values.append(self.compute_kl_to_posterior(mu, sigma))
        
        ax.scatter(elbo_values, kl_values, alpha=0.5, c=kl_values, cmap='coolwarm')
        ax.set_xlabel('ELBO')
        ax.set_ylabel('KL(q || posterior)')
        ax.set_title('ELBO vs KL Divergence\n(Higher ELBO → Lower KL)')
        ax.grid(True, alpha=0.3)
        
        # Plot 4: ELBO decomposition during optimization
        ax = axes[1, 1]
        
        # Simulate optimization path
        q_mus = np.linspace(0, self.posterior_mu, 50)
        q_sigmas = np.linspace(1.0, self.posterior_sigma, 50)
        
        elbos = [self.compute_elbo(m, s) for m, s in zip(q_mus, q_sigmas)]
        kls = [self.compute_kl_to_posterior(m, s) for m, s in zip(q_mus, q_sigmas)]
        
        ax.plot(elbos, 'b-', lw=2, label='ELBO')
        ax.axhline(self.log_evidence, color='green', linestyle='--', 
                   label=f'log p(x) = {self.log_evidence:.2f}')
        ax.fill_between(range(len(elbos)), elbos, self.log_evidence, 
                        alpha=0.3, color='red', label='Gap = KL(q||p)')
        ax.set_xlabel('Optimization Step')
        ax.set_ylabel('Value')
        ax.set_title('ELBO Optimization: log p(x) = ELBO + KL')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 Results:")
        print(f"   True posterior: N({self.posterior_mu:.4f}, {self.posterior_sigma:.4f}²)")
        print(f"   Log evidence: {self.log_evidence:.4f}")
        print(f"   Optimal ELBO: {self.compute_elbo(self.posterior_mu, self.posterior_sigma):.4f}")
        print(f"   Gap (should be ≈0): {self.log_evidence - self.compute_elbo(self.posterior_mu, self.posterior_sigma):.6f}")


# Run visualization
elbo_viz = ELBOVisualization()
elbo_viz.visualize()

---

## 4. Mean-Field Variational Inference <a name="4-mean-field"></a>

The **Mean-Field Approximation** is the most common form of VI. It assumes the variational distribution factorizes over all latent variables:

$$q(z_1, z_2, \ldots, z_K) = \prod_{k=1}^K q_k(z_k)$$

This independence assumption makes optimization tractable but limits the approximation's expressiveness.

### Optimal Update Equations

For mean-field VI, the optimal $q_j^*(z_j)$ that maximizes ELBO is:

$$\log q_j^*(z_j) = \mathbb{E}_{q_{-j}}[\log p(z, x)] + \text{const}$$

Where $q_{-j}$ denotes all factors except $q_j$.

### Application: Bayesian Linear Regression

Consider the model:
- **Likelihood**: $y | X, w, \tau \sim \mathcal{N}(Xw, \tau^{-1}I)$
- **Prior on weights**: $w \sim \mathcal{N}(0, \alpha^{-1}I)$  
- **Prior on precision**: $\tau \sim \text{Gamma}(a_0, b_0)$

We seek: $q(w, \tau) = q(w)q(\tau)$

In [ ]:
class MeanFieldBayesianLinearRegression:
    """
    Mean-Field Variational Inference for Bayesian Linear Regression.
    
    Model:
        y = Xw + ε, ε ~ N(0, τ⁻¹I)
        w ~ N(0, α⁻¹I)
        τ ~ Gamma(a₀, b₀)
    
    Variational family:
        q(w, τ) = q(w)q(τ)
        q(w) = N(μ_w, Σ_w)
        q(τ) = Gamma(a_n, b_n)
    """
    
    def __init__(self, alpha=1.0, a0=1.0, b0=1.0):
        """
        Initialize hyperparameters.
        
        Args:
            alpha: Precision of weight prior
            a0, b0: Parameters of Gamma prior on noise precision
        """
        self.alpha = alpha
        self.a0 = a0
        self.b0 = b0
        self.elbo_history = []
        
    def fit(self, X, y, max_iter=100, tol=1e-6):
        """
        Fit the model using coordinate ascent variational inference.
        
        The update equations are derived from:
        log q*(w) ∝ E_q(τ)[log p(y|X,w,τ)] + log p(w)
        log q*(τ) ∝ E_q(w)[log p(y|X,w,τ)] + log p(τ)
        """
        N, D = X.shape
        
        # Initialize variational parameters
        # q(w) = N(μ_w, Σ_w)
        self.mu_w = np.zeros(D)
        self.Sigma_w = np.eye(D) / self.alpha
        
        # q(τ) = Gamma(a_n, b_n)
        self.a_n = self.a0
        self.b_n = self.b0
        
        # Precompute
        XtX = X.T @ X
        Xty = X.T @ y
        yty = y.T @ y
        
        prev_elbo = -np.inf
        
        for iteration in range(max_iter):
            # E[τ] from current q(τ)
            E_tau = self.a_n / self.b_n
            
            # Update q(w) = N(μ_w, Σ_w)
            # Σ_w = (αI + E[τ]X'X)⁻¹
            # μ_w = E[τ] Σ_w X'y
            self.Sigma_w = np.linalg.inv(self.alpha * np.eye(D) + E_tau * XtX)
            self.mu_w = E_tau * self.Sigma_w @ Xty
            
            # Update q(τ) = Gamma(a_n, b_n)
            # a_n = a₀ + N/2
            # b_n = b₀ + (1/2) E_q(w)[(y - Xw)'(y - Xw)]
            #     = b₀ + (1/2) [y'y - 2μ'_w X'y + tr(X'X(Σ_w + μ_w μ'_w))]
            self.a_n = self.a0 + N / 2
            
            E_wtXtXw = np.trace(XtX @ self.Sigma_w) + self.mu_w @ XtX @ self.mu_w
            self.b_n = self.b0 + 0.5 * (yty - 2 * self.mu_w @ Xty + E_wtXtXw)
            
            # Compute ELBO
            elbo = self._compute_elbo(X, y)
            self.elbo_history.append(elbo)
            
            # Check convergence
            if abs(elbo - prev_elbo) < tol:
                print(f"Converged at iteration {iteration + 1}")
                break
            prev_elbo = elbo
        
        return self
    
    def _compute_elbo(self, X, y):
        """
        Compute the Evidence Lower Bound.
        
        ELBO = E_q[log p(y|X,w,τ)] + E_q[log p(w)] + E_q[log p(τ)]
               - E_q[log q(w)] - E_q[log q(τ)]
        """
        N, D = X.shape
        
        E_tau = self.a_n / self.b_n
        E_log_tau = digamma(self.a_n) - np.log(self.b_n)
        
        # E_q[log p(y|X,w,τ)]
        XtX = X.T @ X
        Xty = X.T @ y
        yty = y.T @ y
        
        E_wtXtXw = np.trace(XtX @ self.Sigma_w) + self.mu_w @ XtX @ self.mu_w
        
        E_log_lik = (N/2 * E_log_tau - N/2 * np.log(2*np.pi) 
                   - E_tau/2 * (yty - 2*self.mu_w @ Xty + E_wtXtXw))
        
        # E_q[log p(w)]
        E_wtw = np.trace(self.Sigma_w) + self.mu_w @ self.mu_w
        E_log_prior_w = (-D/2 * np.log(2*np.pi/self.alpha) - self.alpha/2 * E_wtw)
        
        # E_q[log p(τ)]
        E_log_prior_tau = ((self.a0 - 1) * E_log_tau - self.b0 * E_tau 
                          + self.a0 * np.log(self.b0) - gammaln(self.a0))
        
        # -E_q[log q(w)] = entropy of Gaussian
        H_q_w = 0.5 * D * (1 + np.log(2*np.pi)) + 0.5 * np.linalg.slogdet(self.Sigma_w)[1]
        
        # -E_q[log q(τ)] = entropy of Gamma
        H_q_tau = (self.a_n - np.log(self.b_n) + gammaln(self.a_n) 
                  + (1 - self.a_n) * digamma(self.a_n))
        
        elbo = E_log_lik + E_log_prior_w + E_log_prior_tau + H_q_w + H_q_tau
        return elbo
    
    def predict(self, X_new, return_std=False):
        """
        Predict using the variational posterior.
        
        Returns:
            y_pred: Posterior predictive mean
            y_std: Posterior predictive standard deviation (if return_std=True)
        """
        y_pred = X_new @ self.mu_w
        
        if return_std:
            # Predictive variance = E[τ⁻¹] + X_new Σ_w X_new'
            E_tau_inv = self.b_n / (self.a_n - 1) if self.a_n > 1 else np.inf
            var_pred = E_tau_inv + np.sum(X_new @ self.Sigma_w * X_new, axis=1)
            return y_pred, np.sqrt(var_pred)
        
        return y_pred


# Generate synthetic data for Bayesian Linear Regression
np.random.seed(42)

# True parameters
true_w = np.array([2.0, -1.5, 0.5])
true_tau = 4.0  # Noise precision (variance = 0.25)

# Generate features
N_train = 200
N_test = 50
X_train = np.random.randn(N_train, 3)
X_test = np.random.randn(N_test, 3)

# Generate targets with noise
y_train = X_train @ true_w + np.random.normal(0, 1/np.sqrt(true_tau), N_train)
y_test = X_test @ true_w + np.random.normal(0, 1/np.sqrt(true_tau), N_test)

# Fit Mean-Field VI model
vi_model = MeanFieldBayesianLinearRegression(alpha=1.0, a0=1.0, b0=1.0)
vi_model.fit(X_train, y_train, max_iter=100)

# Predictions
y_pred_train, y_std_train = vi_model.predict(X_train, return_std=True)
y_pred_test, y_std_test = vi_model.predict(X_test, return_std=True)

# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: ELBO convergence
ax = axes[0, 0]
ax.plot(vi_model.elbo_history, 'b-', lw=2)
ax.set_xlabel('Iteration')
ax.set_ylabel('ELBO')
ax.set_title('ELBO Convergence')
ax.grid(True, alpha=0.3)

# Plot 2: Weight posterior
ax = axes[0, 1]
x_range = np.linspace(-3, 4, 500)
colors = ['blue', 'red', 'green']
for i, (true_val, mu, sigma, color) in enumerate(zip(
    true_w, vi_model.mu_w, np.sqrt(np.diag(vi_model.Sigma_w)), colors
)):
    ax.plot(x_range, stats.norm.pdf(x_range, mu, sigma), 
            color=color, lw=2, label=f'q(w_{i}): μ={mu:.2f}, σ={sigma:.2f}')
    ax.axvline(true_val, color=color, linestyle='--', alpha=0.7)

ax.set_xlabel('Weight value')
ax.set_ylabel('Density')
ax.set_title('Posterior Distributions over Weights\n(dashed = true values)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Predictions vs Actuals
ax = axes[1, 0]
ax.errorbar(y_test, y_pred_test, yerr=2*y_std_test, fmt='o', alpha=0.5, 
            capsize=3, label='Predictions ± 2σ')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
        'r--', lw=2, label='Perfect prediction')
ax.set_xlabel('True y')
ax.set_ylabel('Predicted y')
ax.set_title('Test Set: Predictions vs Actuals')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Noise precision posterior
ax = axes[1, 1]
tau_range = np.linspace(0.1, 10, 500)
ax.plot(tau_range, stats.gamma.pdf(tau_range, vi_model.a_n, scale=1/vi_model.b_n), 
        'b-', lw=2, label=f'q(τ): Gamma({vi_model.a_n:.1f}, {vi_model.b_n:.1f})')
ax.axvline(true_tau, color='red', linestyle='--', lw=2, label=f'True τ = {true_tau}')
ax.axvline(vi_model.a_n/vi_model.b_n, color='blue', linestyle=':', lw=2, 
           label=f'E[τ] = {vi_model.a_n/vi_model.b_n:.2f}')
ax.set_xlabel('τ (noise precision)')
ax.set_ylabel('Density')
ax.set_title('Posterior Distribution over Noise Precision')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\n📊 Mean-Field VI Results:")
print(f"   True weights: {true_w}")
print(f"   VI posterior means: {vi_model.mu_w}")
print(f"   VI posterior stds: {np.sqrt(np.diag(vi_model.Sigma_w))}")
print(f"\n   True noise precision: {true_tau}")
print(f"   VI E[τ]: {vi_model.a_n/vi_model.b_n:.3f}")
print(f"\n   Test RMSE: {np.sqrt(np.mean((y_test - y_pred_test)**2)):.4f}")

---

## 5. Variational Inference for Gaussian Mixture Models <a name="5-gmm"></a>

Gaussian Mixture Models (GMMs) are a classic application of VI, where we infer:
- **Cluster assignments** $z_n$ for each data point
- **Cluster means** $\mu_k$
- **Cluster precisions** $\Lambda_k$
- **Mixing proportions** $\pi$

### Model Specification

$$p(x, z, \mu, \Lambda, \pi) = p(\pi) \prod_k p(\mu_k) p(\Lambda_k) \prod_n p(z_n|\pi) p(x_n|z_n, \mu, \Lambda)$$

### Mean-Field Factorization

$$q(z, \mu, \Lambda, \pi) = q(\pi) \prod_k q(\mu_k) q(\Lambda_k) \prod_n q(z_n)$$

This leads to iterative updates similar to EM, but with full posterior distributions.

In [ ]:
class VariationalGMM:
    """
    Variational Inference for Gaussian Mixture Models.
    
    Implements mean-field VI with:
    - Dirichlet prior on mixing proportions
    - Normal-Wishart prior on component parameters
    
    This is similar to sklearn's BayesianGaussianMixture but from scratch.
    """
    
    def __init__(self, n_components=3, alpha_0=1.0, beta_0=1.0, 
                 nu_0=None, m_0=None, W_0=None):
        """
        Initialize prior hyperparameters.
        
        Args:
            n_components: Number of mixture components
            alpha_0: Dirichlet concentration parameter
            beta_0: Scale for mean prior
            nu_0: Degrees of freedom for Wishart (default: D)
            m_0: Prior mean (default: zeros)
            W_0: Prior precision matrix (default: identity)
        """
        self.K = n_components
        self.alpha_0 = alpha_0
        self.beta_0 = beta_0
        self.nu_0 = nu_0
        self.m_0 = m_0
        self.W_0 = W_0
        self.elbo_history = []
        
    def fit(self, X, max_iter=100, tol=1e-6):
        """
        Fit the Variational GMM using coordinate ascent.
        """
        N, D = X.shape
        
        # Set default hyperparameters if not provided
        if self.nu_0 is None:
            self.nu_0 = D
        if self.m_0 is None:
            self.m_0 = np.zeros(D)
        if self.W_0 is None:
            self.W_0 = np.eye(D)
        
        # Initialize variational parameters
        # q(π) = Dir(α), q(μ_k, Λ_k) = NW(m_k, β_k, W_k, ν_k)
        self.alpha = np.ones(self.K) * self.alpha_0
        self.beta = np.ones(self.K) * self.beta_0
        self.nu = np.ones(self.K) * self.nu_0
        self.m = X[np.random.choice(N, self.K, replace=False)]  # Initialize to random data points
        self.W = np.array([self.W_0.copy() for _ in range(self.K)])
        
        # Responsibilities q(z_n = k)
        self.r = np.random.dirichlet(np.ones(self.K), N)
        
        prev_elbo = -np.inf
        
        for iteration in range(max_iter):
            # E-step: Update responsibilities
            self._update_responsibilities(X)
            
            # M-step: Update variational parameters for π, μ, Λ
            self._update_parameters(X)
            
            # Compute ELBO
            elbo = self._compute_elbo(X)
            self.elbo_history.append(elbo)
            
            if abs(elbo - prev_elbo) < tol:
                print(f"Converged at iteration {iteration + 1}")
                break
            prev_elbo = elbo
            
        return self
    
    def _update_responsibilities(self, X):
        """
        Update q(z_n) - the cluster responsibilities.
        
        log r_nk ∝ E[log π_k] + (1/2)E[log|Λ_k|] 
                   - (D/2)log(2π) - (1/2)E[(x_n - μ_k)'Λ_k(x_n - μ_k)]
        """
        N, D = X.shape
        
        # E[log π_k] from Dirichlet
        E_log_pi = digamma(self.alpha) - digamma(np.sum(self.alpha))
        
        # E[log |Λ_k|] from Wishart
        E_log_det_Lambda = np.zeros(self.K)
        for k in range(self.K):
            E_log_det_Lambda[k] = (D * np.log(2) + np.linalg.slogdet(self.W[k])[1] +
                                  np.sum(digamma((self.nu[k] + 1 - np.arange(1, D+1))/2)))
        
        # E[(x - μ)'Λ(x - μ)]
        log_rho = np.zeros((N, self.K))
        for k in range(self.K):
            diff = X - self.m[k]
            # E[(x-μ)'Λ(x-μ)] = D/β + ν*(x-m)'W(x-m)
            E_quad = D/self.beta[k] + self.nu[k] * np.sum(diff @ self.W[k] * diff, axis=1)
            log_rho[:, k] = E_log_pi[k] + 0.5*E_log_det_Lambda[k] - D/2*np.log(2*np.pi) - 0.5*E_quad
        
        # Normalize responsibilities
        log_rho_max = np.max(log_rho, axis=1, keepdims=True)
        self.r = np.exp(log_rho - log_rho_max)
        self.r /= np.sum(self.r, axis=1, keepdims=True)
        
        # Statistics
        self.N_k = np.sum(self.r, axis=0) + 1e-10  # Avoid division by zero
        self.x_bar = (self.r.T @ X) / self.N_k[:, np.newaxis]
        
    def _update_parameters(self, X):
        """
        Update variational parameters for π and (μ, Λ).
        """
        N, D = X.shape
        
        # Update Dirichlet parameters
        self.alpha = self.alpha_0 + self.N_k
        
        # Update Normal-Wishart parameters for each component
        for k in range(self.K):
            # Sufficient statistics
            x_bar_k = self.x_bar[k]
            diff = X - x_bar_k
            S_k = (self.r[:, k:k+1] * diff).T @ diff / self.N_k[k]
            
            # Update parameters
            self.beta[k] = self.beta_0 + self.N_k[k]
            self.m[k] = (self.beta_0 * self.m_0 + self.N_k[k] * x_bar_k) / self.beta[k]
            self.nu[k] = self.nu_0 + self.N_k[k]
            
            diff_m = x_bar_k - self.m_0
            self.W[k] = np.linalg.inv(
                np.linalg.inv(self.W_0) + self.N_k[k] * S_k +
                (self.beta_0 * self.N_k[k] / self.beta[k]) * np.outer(diff_m, diff_m)
            )
    
    def _compute_elbo(self, X):
        """
        Compute the Evidence Lower Bound.
        """
        N, D = X.shape
        
        # This is a simplified ELBO computation
        # Full computation involves multiple expectation terms
        
        # E[log p(X|Z,μ,Λ)]
        E_log_lik = 0
        for k in range(self.K):
            E_log_det = D*np.log(2) + np.linalg.slogdet(self.W[k])[1] + \
                       np.sum(digamma((self.nu[k]+1-np.arange(1,D+1))/2))
            diff = X - self.m[k]
            E_quad = D/self.beta[k] + self.nu[k] * np.sum(diff @ self.W[k] * diff, axis=1)
            E_log_lik += np.sum(self.r[:, k] * (0.5*E_log_det - D/2*np.log(2*np.pi) - 0.5*E_quad))
        
        # E[log p(Z|π)]
        E_log_pi = digamma(self.alpha) - digamma(np.sum(self.alpha))
        E_log_pz = np.sum(self.r * E_log_pi)
        
        # -E[log q(Z)]
        H_qz = -np.sum(self.r * np.log(self.r + 1e-10))
        
        elbo = E_log_lik + E_log_pz + H_qz
        return elbo
    
    def predict(self, X):
        """Return cluster assignments."""
        self._update_responsibilities(X)
        return np.argmax(self.r, axis=1)
    
    def predict_proba(self, X):
        """Return cluster probabilities."""
        self._update_responsibilities(X)
        return self.r


# Generate synthetic data with 3 clusters
np.random.seed(42)
n_samples = 500

# True cluster parameters
true_means = [[-3, -3], [0, 4], [4, 0]]
true_covs = [[[0.5, 0.2], [0.2, 0.5]], 
             [[1.0, 0], [0, 0.3]], 
             [[0.3, -0.1], [-0.1, 0.8]]]
true_weights = [0.3, 0.4, 0.3]

# Sample from mixture
X_gmm = []
true_labels = []
for i, (mean, cov, weight) in enumerate(zip(true_means, true_covs, true_weights)):
    n = int(n_samples * weight)
    X_gmm.append(np.random.multivariate_normal(mean, cov, n))
    true_labels.extend([i] * n)

X_gmm = np.vstack(X_gmm)
true_labels = np.array(true_labels)

# Shuffle
shuffle_idx = np.random.permutation(len(X_gmm))
X_gmm = X_gmm[shuffle_idx]
true_labels = true_labels[shuffle_idx]

# Fit Variational GMM
vgmm = VariationalGMM(n_components=3)
vgmm.fit(X_gmm, max_iter=100)

# Get cluster assignments
pred_labels = vgmm.predict(X_gmm)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: True clusters
ax = axes[0]
scatter = ax.scatter(X_gmm[:, 0], X_gmm[:, 1], c=true_labels, cmap='viridis', alpha=0.6, s=20)
for mean in true_means:
    ax.scatter(*mean, c='red', marker='x', s=200, linewidths=3)
ax.set_title('True Clusters')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')

# Plot 2: VI inferred clusters
ax = axes[1]
ax.scatter(X_gmm[:, 0], X_gmm[:, 1], c=pred_labels, cmap='viridis', alpha=0.6, s=20)
for k in range(vgmm.K):
    ax.scatter(*vgmm.m[k], c='red', marker='*', s=200, linewidths=2)
    # Draw covariance ellipse
    cov_k = np.linalg.inv(vgmm.nu[k] * vgmm.W[k])  # Expected covariance
    eigenvalues, eigenvectors = np.linalg.eigh(cov_k)
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
    width, height = 2 * 2 * np.sqrt(eigenvalues)  # 2 std
    ellipse = Ellipse(vgmm.m[k], width, height, angle=angle, 
                     fill=False, color='red', linewidth=2)
    ax.add_patch(ellipse)
ax.set_title('VI Inferred Clusters')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')

# Plot 3: ELBO convergence
ax = axes[2]
ax.plot(vgmm.elbo_history, 'b-', lw=2)
ax.set_xlabel('Iteration')
ax.set_ylabel('ELBO')
ax.set_title('ELBO Convergence')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print mixing proportions
print("\n📊 Variational GMM Results:")
print(f"   True mixing proportions: {true_weights}")
print(f"   VI posterior E[π]: {vgmm.alpha / np.sum(vgmm.alpha)}")
print(f"\n   Effective components (N_k > 1): {np.sum(vgmm.N_k > 1)}")

---

## 6. Coordinate Ascent Variational Inference (CAVI) <a name="6-cavi"></a>

**Coordinate Ascent Variational Inference (CAVI)** is a systematic algorithm for optimizing the ELBO by iteratively updating each variational factor while holding others fixed.

### CAVI Algorithm

For mean-field factorization $q(z) = \prod_j q_j(z_j)$:

**repeat until convergence:**
- **for** each latent variable $j$:
  $$\log q_j^*(z_j) \leftarrow \mathbb{E}_{-j}[\log p(z, x)] + \text{const}$$

The key insight is that the optimal form of each factor is determined by the expected complete conditional.

### Properties

- **Monotonic increase**: Each update is guaranteed to increase (or maintain) the ELBO
- **Coordinate-wise**: Only one factor changes at a time
- **Closed-form**: For conjugate exponential family models, updates have analytical solutions

In [ ]:
class CAVI_BayesianFactorModel:
    """
    Coordinate Ascent VI for a simple Bayesian Factor Model.
    
    Model: Asset returns decompose into factor exposures and idiosyncratic risk
        r_t = B @ f_t + ε_t
        
    Where:
        r_t: N-dimensional asset returns at time t
        B: N × K factor loading matrix
        f_t: K-dimensional latent factors
        ε_t ~ N(0, Ψ) idiosyncratic noise
    
    Priors:
        B_nk ~ N(0, σ_B²)
        f_tk ~ N(0, 1)
        ψ_n ~ InvGamma(a, b)
    
    This is a simplified factor model for demonstration.
    """
    
    def __init__(self, n_factors=3, sigma_B=1.0, a_psi=1.0, b_psi=1.0):
        self.K = n_factors
        self.sigma_B = sigma_B
        self.a_psi = a_psi
        self.b_psi = b_psi
        self.elbo_history = []
        
    def fit(self, R, max_iter=100, tol=1e-6):
        """
        Fit the factor model using CAVI.
        
        Args:
            R: T × N matrix of returns (T time periods, N assets)
        """
        T, N = R.shape
        K = self.K
        
        # Initialize variational parameters
        # q(B) = N(m_B, S_B) - each row independently
        self.m_B = np.random.randn(N, K) * 0.1
        self.S_B = np.array([np.eye(K) for _ in range(N)])
        
        # q(F) = N(m_F, S_F) - each row independently
        self.m_F = np.random.randn(T, K) * 0.1
        self.S_F = np.array([np.eye(K) for _ in range(T)])
        
        # q(ψ) = InvGamma(a_n, b_n)
        self.a_n = np.ones(N) * (self.a_psi + T/2)
        self.b_n = np.ones(N) * self.b_psi
        
        prev_elbo = -np.inf
        
        for iteration in range(max_iter):
            # CAVI updates
            self._update_factors(R)      # Update q(F)
            self._update_loadings(R)     # Update q(B)
            self._update_precisions(R)   # Update q(ψ)
            
            # Compute ELBO
            elbo = self._compute_elbo(R)
            self.elbo_history.append(elbo)
            
            if iteration > 0 and abs(elbo - prev_elbo) < tol:
                print(f"CAVI converged at iteration {iteration + 1}")
                break
            prev_elbo = elbo
            
        return self
    
    def _update_factors(self, R):
        """Update q(F) = ∏_t q(f_t)."""
        T, N = R.shape
        
        # E[Ψ⁻¹] diagonal
        E_psi_inv = self.a_n / self.b_n
        
        for t in range(T):
            # S_F_t = (I + E[B'Ψ⁻¹B])⁻¹
            E_BtPsiB = np.zeros((self.K, self.K))
            E_BtPsi_r = np.zeros(self.K)
            
            for n in range(N):
                # E[B_n' Ψ_n⁻¹ B_n] = Ψ_n⁻¹ (S_B_n + m_B_n m_B_n')
                E_BtPsiB += E_psi_inv[n] * (self.S_B[n] + np.outer(self.m_B[n], self.m_B[n]))
                E_BtPsi_r += E_psi_inv[n] * self.m_B[n] * R[t, n]
            
            self.S_F[t] = np.linalg.inv(np.eye(self.K) + E_BtPsiB)
            self.m_F[t] = self.S_F[t] @ E_BtPsi_r
    
    def _update_loadings(self, R):
        """Update q(B) = ∏_n q(B_n)."""
        T, N = R.shape
        
        E_psi_inv = self.a_n / self.b_n
        
        # E[F'F] = Σ_t (S_F_t + m_F_t m_F_t')
        E_FtF = np.sum([self.S_F[t] + np.outer(self.m_F[t], self.m_F[t]) for t in range(T)], axis=0)
        
        for n in range(N):
            # S_B_n = (1/σ_B² I + ψ_n⁻¹ E[F'F])⁻¹
            self.S_B[n] = np.linalg.inv(np.eye(self.K)/self.sigma_B**2 + E_psi_inv[n] * E_FtF)
            
            # m_B_n = S_B_n ψ_n⁻¹ E[F]' r_n
            self.m_B[n] = E_psi_inv[n] * self.S_B[n] @ self.m_F.T @ R[:, n]
    
    def _update_precisions(self, R):
        """Update q(ψ_n) = InvGamma(a_n, b_n)."""
        T, N = R.shape
        
        self.a_n = np.ones(N) * (self.a_psi + T/2)
        
        for n in range(N):
            # E[(r_n - B_n f)'(r_n - B_n f)] = Σ_t E[(r_nt - B_n f_t)²]
            expected_sq_error = 0
            for t in range(T):
                # E[(r - Bf)²] = r² - 2r*m_B'm_F + E[f'B'Bf]
                # E[f'B'Bf] = tr(E[BB']E[ff']) = tr((S_B + m_Bm_B')(S_F + m_Fm_F'))
                E_BB = self.S_B[n] + np.outer(self.m_B[n], self.m_B[n])
                E_ff = self.S_F[t] + np.outer(self.m_F[t], self.m_F[t])
                
                expected_sq_error += (R[t, n]**2 
                                     - 2 * R[t, n] * self.m_B[n] @ self.m_F[t]
                                     + np.trace(E_BB @ E_ff))
            
            self.b_n[n] = self.b_psi + 0.5 * expected_sq_error
    
    def _compute_elbo(self, R):
        """Compute ELBO (simplified version)."""
        T, N = R.shape
        
        E_psi_inv = self.a_n / self.b_n
        E_log_psi_inv = digamma(self.a_n) - np.log(self.b_n)
        
        # Expected log likelihood
        elbo = 0.5 * T * np.sum(E_log_psi_inv) - T * N / 2 * np.log(2*np.pi)
        
        for n in range(N):
            for t in range(T):
                E_BB = self.S_B[n] + np.outer(self.m_B[n], self.m_B[n])
                E_ff = self.S_F[t] + np.outer(self.m_F[t], self.m_F[t])
                expected_sq = (R[t, n]**2 - 2*R[t, n]*self.m_B[n]@self.m_F[t] + np.trace(E_BB @ E_ff))
                elbo -= 0.5 * E_psi_inv[n] * expected_sq
        
        # Add entropy terms (simplified)
        for n in range(N):
            elbo += 0.5 * np.linalg.slogdet(self.S_B[n])[1]
        for t in range(T):
            elbo += 0.5 * np.linalg.slogdet(self.S_F[t])[1]
        
        return elbo
    
    def get_factor_loadings(self):
        """Return posterior mean of factor loadings."""
        return self.m_B
    
    def get_factors(self):
        """Return posterior mean of latent factors."""
        return self.m_F


# Generate synthetic factor model data
np.random.seed(42)

# True parameters
T, N, K_true = 250, 10, 3  # 250 days, 10 assets, 3 factors

# True factor loadings (sparse structure)
B_true = np.array([
    [1.0, 0.2, 0.0],   # Asset 1: High market beta
    [0.8, 0.3, 0.1],   # Asset 2
    [0.6, 0.7, 0.0],   # Asset 3: Value factor
    [0.5, 0.5, 0.5],   # Asset 4: Balanced
    [0.3, 0.1, 0.9],   # Asset 5: Momentum factor
    [0.9, 0.0, 0.2],   # Asset 6
    [0.4, 0.8, 0.1],   # Asset 7
    [0.7, 0.3, 0.4],   # Asset 8
    [0.2, 0.6, 0.6],   # Asset 9
    [0.5, 0.4, 0.3],   # Asset 10
])

# True latent factors (standard normal)
F_true = np.random.randn(T, K_true)

# Idiosyncratic noise
psi_true = np.array([0.05, 0.08, 0.04, 0.06, 0.07, 0.05, 0.09, 0.06, 0.08, 0.05])
epsilon = np.random.randn(T, N) * np.sqrt(psi_true)

# Observed returns
R_observed = F_true @ B_true.T + epsilon

# Fit CAVI factor model
cavi_model = CAVI_BayesianFactorModel(n_factors=3)
cavi_model.fit(R_observed, max_iter=100)

# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: ELBO convergence
ax = axes[0, 0]
ax.plot(cavi_model.elbo_history, 'b-', lw=2)
ax.set_xlabel('CAVI Iteration')
ax.set_ylabel('ELBO')
ax.set_title('CAVI Convergence for Factor Model')
ax.grid(True, alpha=0.3)

# Plot 2: Recovered factor loadings
ax = axes[0, 1]
B_recovered = cavi_model.get_factor_loadings()
# Need to handle factor ordering/sign ambiguity
for k in range(K_true):
    if np.corrcoef(B_true[:, k], B_recovered[:, k])[0, 1] < 0:
        B_recovered[:, k] *= -1

im = ax.imshow(np.hstack([B_true, B_recovered]), aspect='auto', cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xticks([0, 1, 2, 4, 5, 6])
ax.set_xticklabels(['F1', 'F2', 'F3', 'F1', 'F2', 'F3'])
ax.axvline(2.5, color='black', lw=2)
ax.set_ylabel('Asset')
ax.set_title('Factor Loadings: True (left) vs Recovered (right)')
plt.colorbar(im, ax=ax, label='Loading')

# Plot 3: Recovered factors
ax = axes[1, 0]
F_recovered = cavi_model.get_factors()
for k in range(K_true):
    if np.corrcoef(F_true[:, k], F_recovered[:, k])[0, 1] < 0:
        F_recovered[:, k] *= -1
        
ax.plot(F_true[:50, 0], 'b-', alpha=0.7, label='True F1')
ax.plot(F_recovered[:50, 0], 'r--', alpha=0.7, label='Recovered F1')
ax.set_xlabel('Time')
ax.set_ylabel('Factor Value')
ax.set_title('Latent Factor Recovery (First 50 periods)')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Idiosyncratic variance recovery
ax = axes[1, 1]
psi_recovered = cavi_model.b_n / (cavi_model.a_n - 1)  # E[ψ] for InvGamma
x_pos = np.arange(N)
width = 0.35
ax.bar(x_pos - width/2, psi_true, width, label='True ψ', alpha=0.7)
ax.bar(x_pos + width/2, psi_recovered, width, label='Recovered E[ψ]', alpha=0.7)
ax.set_xlabel('Asset')
ax.set_ylabel('Idiosyncratic Variance')
ax.set_title('Idiosyncratic Variance Recovery')
ax.legend()
ax.set_xticks(x_pos)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 CAVI Factor Model Results:")
print(f"   Factor loading correlation (per factor):")
for k in range(K_true):
    corr = np.abs(np.corrcoef(B_true[:, k], cavi_model.m_B[:, k])[0, 1])
    print(f"      Factor {k+1}: {corr:.4f}")
print(f"\n   Mean idiosyncratic variance error: {np.mean(np.abs(psi_true - psi_recovered)):.6f}")

---

## 7. Stochastic Variational Inference (SVI) <a name="7-svi"></a>

**Stochastic Variational Inference** enables VI to scale to massive datasets by using:
1. **Mini-batches** instead of full data
2. **Stochastic gradient descent** on the ELBO
3. **Natural gradients** for faster convergence

### The Problem with Batch VI

For $N$ data points:
$$\nabla_\lambda \mathcal{L} = \sum_{i=1}^N \nabla_\lambda \mathbb{E}_q[\log p(x_i, z)] - \nabla_\lambda \mathbb{E}_q[\log q(z)]$$

Computing gradients requires a pass over all data — $O(N)$ per iteration!

### SVI Solution

Use an unbiased estimate with mini-batch $\mathcal{B}$:
$$\hat{\nabla}_\lambda \mathcal{L} = \frac{N}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \nabla_\lambda \mathbb{E}_q[\log p(x_i, z)] - \nabla_\lambda \mathbb{E}_q[\log q(z)]$$

### Natural Gradients

Standard gradients ignore the geometry of probability distributions. **Natural gradients** account for the Riemannian geometry:

$$\tilde{\nabla}_\lambda \mathcal{L} = F^{-1} \nabla_\lambda \mathcal{L}$$

Where $F$ is the Fisher information matrix. For exponential families, this simplifies beautifully!

In [ ]:
class StochasticVI_BayesianLogistic:
    """
    Stochastic Variational Inference for Bayesian Logistic Regression.
    
    Model:
        y | x, w ~ Bernoulli(sigmoid(w'x))
        w ~ N(0, σ²I)
    
    Uses:
        - Mini-batch gradient estimates
        - Reparameterization trick for gradients
        - Adam optimizer
    """
    
    def __init__(self, learning_rate=0.01, prior_var=1.0):
        self.lr = learning_rate
        self.prior_var = prior_var
        self.elbo_history = []
        
    def fit(self, X, y, batch_size=64, n_epochs=100, n_samples=10):
        """
        Fit using SVI with mini-batches.
        
        Args:
            X: Features (N × D)
            y: Binary labels (N,)
            batch_size: Mini-batch size
            n_epochs: Number of epochs
            n_samples: Monte Carlo samples for gradient estimation
        """
        N, D = X.shape
        
        # Initialize variational parameters: q(w) = N(μ, diag(σ²))
        self.mu = np.zeros(D)
        self.log_sigma = np.zeros(D)  # log(σ) for stability
        
        # Adam optimizer state
        m_mu, v_mu = np.zeros(D), np.zeros(D)
        m_logsig, v_logsig = np.zeros(D), np.zeros(D)
        beta1, beta2, eps = 0.9, 0.999, 1e-8
        t = 0
        
        n_batches = N // batch_size
        
        for epoch in range(n_epochs):
            # Shuffle data
            perm = np.random.permutation(N)
            X_shuffled = X[perm]
            y_shuffled = y[perm]
            
            epoch_elbo = 0
            
            for batch_idx in range(n_batches):
                t += 1
                start = batch_idx * batch_size
                end = start + batch_size
                X_batch = X_shuffled[start:end]
                y_batch = y_shuffled[start:end]
                
                # Compute stochastic gradients
                grad_mu, grad_logsig, batch_elbo = self._compute_gradients(
                    X_batch, y_batch, N, n_samples
                )
                epoch_elbo += batch_elbo
                
                # Adam updates for mu
                m_mu = beta1 * m_mu + (1 - beta1) * grad_mu
                v_mu = beta2 * v_mu + (1 - beta2) * grad_mu**2
                m_hat = m_mu / (1 - beta1**t)
                v_hat = v_mu / (1 - beta2**t)
                self.mu += self.lr * m_hat / (np.sqrt(v_hat) + eps)
                
                # Adam updates for log_sigma
                m_logsig = beta1 * m_logsig + (1 - beta1) * grad_logsig
                v_logsig = beta2 * v_logsig + (1 - beta2) * grad_logsig**2
                m_hat = m_logsig / (1 - beta1**t)
                v_hat = v_logsig / (1 - beta2**t)
                self.log_sigma += self.lr * m_hat / (np.sqrt(v_hat) + eps)
            
            self.elbo_history.append(epoch_elbo / n_batches)
            
            if epoch % 20 == 0:
                print(f"Epoch {epoch}: ELBO = {self.elbo_history[-1]:.4f}")
        
        return self
    
    def _compute_gradients(self, X_batch, y_batch, N, n_samples):
        """
        Compute stochastic gradients using reparameterization trick.
        
        The reparameterization trick:
            w = μ + σ ⊙ ε,  where ε ~ N(0, I)
        
        Allows us to compute ∇_μ,σ E_q[f(w)] = E_ε[∇_μ,σ f(μ + σε)]
        """
        M = len(y_batch)
        D = len(self.mu)
        sigma = np.exp(self.log_sigma)
        
        grad_mu = np.zeros(D)
        grad_logsig = np.zeros(D)
        total_elbo = 0
        
        for _ in range(n_samples):
            # Reparameterization: w = μ + σ ⊙ ε
            eps = np.random.randn(D)
            w = self.mu + sigma * eps
            
            # Forward pass
            logits = X_batch @ w
            probs = 1 / (1 + np.exp(-np.clip(logits, -500, 500)))
            
            # Log likelihood gradient (scaled for full dataset)
            log_lik_grad_w = (N / M) * X_batch.T @ (y_batch - probs)
            
            # KL gradient (prior is N(0, σ²_prior I))
            kl_grad_w = -w / self.prior_var
            
            # Total gradient w.r.t. w
            grad_w = log_lik_grad_w + kl_grad_w
            
            # Backprop through reparameterization
            grad_mu += grad_w
            grad_logsig += grad_w * eps * sigma  # Chain rule: ∂w/∂log_σ = σε
            
            # ELBO contribution
            log_lik = np.sum(y_batch * np.log(probs + 1e-10) + 
                           (1 - y_batch) * np.log(1 - probs + 1e-10))
            log_prior = -0.5 * np.sum(w**2) / self.prior_var
            log_q = -0.5 * np.sum(eps**2)  # Entropy term cancels in expectation
            total_elbo += (N / M) * log_lik + log_prior
        
        # Average over samples
        grad_mu /= n_samples
        grad_logsig /= n_samples
        total_elbo /= n_samples
        
        # Add entropy gradient: ∂H[q]/∂log_σ = 1 (since H = D/2(1 + log(2π)) + Σlog(σ))
        grad_logsig += 1.0
        
        return grad_mu, grad_logsig, total_elbo
    
    def predict_proba(self, X, n_samples=100):
        """
        Predict probabilities with uncertainty.
        """
        sigma = np.exp(self.log_sigma)
        probs = np.zeros(len(X))
        
        for _ in range(n_samples):
            w = self.mu + sigma * np.random.randn(len(self.mu))
            logits = X @ w
            probs += 1 / (1 + np.exp(-np.clip(logits, -500, 500)))
        
        return probs / n_samples
    
    def predict(self, X):
        """Predict class labels."""
        return (self.predict_proba(X) > 0.5).astype(int)


# Generate synthetic classification data
np.random.seed(42)

# Large dataset to demonstrate scalability
N_large = 10000
D = 20

# True weights (sparse)
w_true = np.zeros(D)
w_true[:5] = np.array([2.0, -1.5, 1.0, -0.5, 0.8])

X_large = np.random.randn(N_large, D)
logits_true = X_large @ w_true
y_large = (np.random.rand(N_large) < 1/(1 + np.exp(-logits_true))).astype(int)

# Split train/test
train_idx = np.random.choice(N_large, int(0.8 * N_large), replace=False)
test_idx = np.setdiff1d(np.arange(N_large), train_idx)

X_train, y_train = X_large[train_idx], y_large[train_idx]
X_test, y_test = X_large[test_idx], y_large[test_idx]

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Fit SVI model
import time

start_time = time.time()
svi_model = StochasticVI_BayesianLogistic(learning_rate=0.05, prior_var=10.0)
svi_model.fit(X_train, y_train, batch_size=128, n_epochs=100, n_samples=5)
svi_time = time.time() - start_time

# Evaluate
y_pred_proba = svi_model.predict_proba(X_test)
y_pred = svi_model.predict(X_test)
accuracy = np.mean(y_pred == y_test)

print(f"\nSVI completed in {svi_time:.2f} seconds")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
# Visualize SVI results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: ELBO convergence
ax = axes[0, 0]
ax.plot(svi_model.elbo_history, 'b-', lw=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('ELBO (mini-batch average)')
ax.set_title('SVI ELBO Convergence')
ax.grid(True, alpha=0.3)

# Plot 2: Weight recovery
ax = axes[0, 1]
sigma = np.exp(svi_model.log_sigma)
x_pos = np.arange(D)
ax.bar(x_pos, w_true, alpha=0.5, label='True weights', color='blue')
ax.errorbar(x_pos, svi_model.mu, yerr=2*sigma, fmt='o', color='red', 
            capsize=3, label='VI posterior ± 2σ')
ax.set_xlabel('Weight index')
ax.set_ylabel('Weight value')
ax.set_title('Weight Recovery: True vs VI Posterior')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='--', alpha=0.3)

# Plot 3: Uncertainty vs correctness
ax = axes[1, 0]
entropy = -y_pred_proba * np.log(y_pred_proba + 1e-10) - (1-y_pred_proba) * np.log(1-y_pred_proba + 1e-10)
correct = (y_pred == y_test)

# Bin by entropy
n_bins = 10
bins = np.linspace(0, np.log(2), n_bins + 1)
bin_indices = np.digitize(entropy, bins)

accuracy_by_entropy = []
entropy_centers = []
for i in range(1, n_bins + 1):
    mask = bin_indices == i
    if np.sum(mask) > 0:
        accuracy_by_entropy.append(np.mean(correct[mask]))
        entropy_centers.append((bins[i-1] + bins[i]) / 2)

ax.bar(entropy_centers, accuracy_by_entropy, width=0.05, alpha=0.7)
ax.set_xlabel('Predictive Entropy')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy vs Predictive Uncertainty\n(Lower entropy → More confident → Higher accuracy)')
ax.grid(True, alpha=0.3)

# Plot 4: Calibration plot
ax = axes[1, 1]
n_bins = 10
bin_edges = np.linspace(0, 1, n_bins + 1)
bin_indices = np.digitize(y_pred_proba, bin_edges)

mean_predicted = []
fraction_positive = []
bin_counts = []

for i in range(1, n_bins + 1):
    mask = bin_indices == i
    if np.sum(mask) > 0:
        mean_predicted.append(np.mean(y_pred_proba[mask]))
        fraction_positive.append(np.mean(y_test[mask]))
        bin_counts.append(np.sum(mask))

ax.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
ax.scatter(mean_predicted, fraction_positive, s=100, c='red', alpha=0.7, label='SVI')
ax.set_xlabel('Mean Predicted Probability')
ax.set_ylabel('Fraction of Positives')
ax.set_title('Calibration Plot')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 SVI Results Summary:")
print(f"   Dataset size: {N_large} samples")
print(f"   Training time: {svi_time:.2f} seconds")
print(f"   Test accuracy: {accuracy:.4f}")
print(f"   Mean posterior std: {np.mean(sigma):.4f}")
print(f"   Weight recovery correlation: {np.corrcoef(w_true, svi_model.mu)[0,1]:.4f}")

---

## 8. Automatic Differentiation Variational Inference (ADVI) <a name="8-advi"></a>

**ADVI** automates variational inference for any differentiable probabilistic model by:

1. **Transforming** constrained parameters to unconstrained space
2. **Applying** a Gaussian variational family in the transformed space
3. **Using** automatic differentiation to compute ELBO gradients
4. **Optimizing** with standard gradient-based methods

### The Reparameterization Trick

The key innovation enabling gradient-based VI is the **reparameterization trick**:

Instead of sampling from $q_\phi(z)$, we write:
$$z = g(\epsilon, \phi) \quad \text{where} \quad \epsilon \sim p(\epsilon)$$

For Gaussians: $z = \mu + \sigma \odot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$

This allows gradients to flow through the sampling process:
$$\nabla_\phi \mathbb{E}_{q_\phi}[f(z)] = \nabla_\phi \mathbb{E}_{p(\epsilon)}[f(g(\epsilon, \phi))] = \mathbb{E}_{p(\epsilon)}[\nabla_\phi f(g(\epsilon, \phi))]$$

### Transformation to Unconstrained Space

| Parameter Constraint | Transformation |
|---------------------|----------------|
| $\theta \in \mathbb{R}$ | Identity |
| $\theta > 0$ | $\log(\theta) \in \mathbb{R}$ |
| $\theta \in (0, 1)$ | $\text{logit}(\theta) \in \mathbb{R}$ |
| $\theta \in \Delta^K$ (simplex) | Stick-breaking |

In [ ]:
if TORCH_AVAILABLE:
    class ADVI_BayesianNN(nn.Module):
        """
        Automatic Differentiation VI for a Bayesian Neural Network.
        
        Uses the reparameterization trick with diagonal Gaussian posteriors
        over all weights and biases.
        """
        
        def __init__(self, input_dim, hidden_dim=32, output_dim=1, prior_std=1.0):
            super().__init__()
            
            self.prior_std = prior_std
            
            # Variational parameters for first layer
            self.w1_mu = nn.Parameter(torch.randn(input_dim, hidden_dim) * 0.1)
            self.w1_logsigma = nn.Parameter(torch.zeros(input_dim, hidden_dim) - 2)
            self.b1_mu = nn.Parameter(torch.zeros(hidden_dim))
            self.b1_logsigma = nn.Parameter(torch.zeros(hidden_dim) - 2)
            
            # Variational parameters for second layer
            self.w2_mu = nn.Parameter(torch.randn(hidden_dim, output_dim) * 0.1)
            self.w2_logsigma = nn.Parameter(torch.zeros(hidden_dim, output_dim) - 2)
            self.b2_mu = nn.Parameter(torch.zeros(output_dim))
            self.b2_logsigma = nn.Parameter(torch.zeros(output_dim) - 2)
            
            # Observation noise (learned)
            self.log_noise_std = nn.Parameter(torch.tensor(0.0))
        
        def sample_weights(self):
            """Sample weights using reparameterization trick."""
            eps_w1 = torch.randn_like(self.w1_mu)
            eps_b1 = torch.randn_like(self.b1_mu)
            eps_w2 = torch.randn_like(self.w2_mu)
            eps_b2 = torch.randn_like(self.b2_mu)
            
            w1 = self.w1_mu + torch.exp(self.w1_logsigma) * eps_w1
            b1 = self.b1_mu + torch.exp(self.b1_logsigma) * eps_b1
            w2 = self.w2_mu + torch.exp(self.w2_logsigma) * eps_w2
            b2 = self.b2_mu + torch.exp(self.b2_logsigma) * eps_b2
            
            return w1, b1, w2, b2
        
        def forward(self, x, w1, b1, w2, b2):
            """Forward pass with given weights."""
            h = torch.tanh(x @ w1 + b1)
            out = h @ w2 + b2
            return out
        
        def kl_divergence(self):
            """Compute KL(q || prior) for all parameters."""
            kl = 0.0
            
            # KL for each parameter group
            for mu, logsigma in [(self.w1_mu, self.w1_logsigma),
                                  (self.b1_mu, self.b1_logsigma),
                                  (self.w2_mu, self.w2_logsigma),
                                  (self.b2_mu, self.b2_logsigma)]:
                sigma = torch.exp(logsigma)
                # KL(N(μ,σ²) || N(0, prior_std²))
                kl += 0.5 * torch.sum(
                    (sigma**2 + mu**2) / self.prior_std**2 
                    - 1 
                    - 2*logsigma 
                    + 2*np.log(self.prior_std)
                )
            
            return kl
        
        def elbo(self, x, y, n_samples=5):
            """Compute ELBO using Monte Carlo estimate."""
            batch_size = len(x)
            log_lik = 0.0
            noise_std = torch.exp(self.log_noise_std)
            
            for _ in range(n_samples):
                w1, b1, w2, b2 = self.sample_weights()
                pred = self.forward(x, w1, b1, w2, b2)
                
                # Log likelihood: y ~ N(pred, noise_std²)
                log_lik += torch.sum(
                    -0.5 * np.log(2*np.pi) 
                    - self.log_noise_std 
                    - 0.5 * ((y - pred) / noise_std)**2
                )
            
            log_lik /= n_samples
            kl = self.kl_divergence()
            
            return log_lik - kl
        
        def predict(self, x, n_samples=100):
            """Predict with uncertainty quantification."""
            preds = []
            
            with torch.no_grad():
                for _ in range(n_samples):
                    w1, b1, w2, b2 = self.sample_weights()
                    pred = self.forward(x, w1, b1, w2, b2)
                    preds.append(pred.numpy())
            
            preds = np.array(preds)
            return preds.mean(axis=0), preds.std(axis=0)

    
    # Generate nonlinear regression data
    np.random.seed(42)
    torch.manual_seed(42)
    
    N = 500
    X_nl = np.random.uniform(-3, 3, (N, 1)).astype(np.float32)
    y_true_func = np.sin(X_nl) + 0.3 * np.cos(3*X_nl)  # Nonlinear function
    y_nl = y_true_func + np.random.normal(0, 0.2, (N, 1)).astype(np.float32)  # Add noise
    
    # Train/test split
    train_mask = np.random.rand(N) < 0.8
    X_train_nl = torch.tensor(X_nl[train_mask])
    y_train_nl = torch.tensor(y_nl[train_mask])
    X_test_nl = torch.tensor(X_nl[~train_mask])
    y_test_nl = torch.tensor(y_nl[~train_mask])
    
    # Train ADVI BNN
    bnn = ADVI_BayesianNN(input_dim=1, hidden_dim=50, output_dim=1, prior_std=1.0)
    optimizer = optim.Adam(bnn.parameters(), lr=0.01)
    
    elbo_history_bnn = []
    
    print("Training ADVI Bayesian Neural Network...")
    for epoch in range(500):
        optimizer.zero_grad()
        loss = -bnn.elbo(X_train_nl, y_train_nl, n_samples=5)
        loss.backward()
        optimizer.step()
        elbo_history_bnn.append(-loss.item())
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: ELBO = {-loss.item():.4f}")
    
    # Predictions
    X_plot = torch.tensor(np.linspace(-4, 4, 200).reshape(-1, 1).astype(np.float32))
    y_mean, y_std = bnn.predict(X_plot, n_samples=200)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    ax = axes[0]
    ax.plot(elbo_history_bnn, 'b-', lw=1)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('ELBO')
    ax.set_title('ADVI BNN Training: ELBO Convergence')
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    ax.scatter(X_train_nl.numpy(), y_train_nl.numpy(), alpha=0.3, s=10, label='Training data')
    ax.scatter(X_test_nl.numpy(), y_test_nl.numpy(), alpha=0.5, s=20, c='green', label='Test data')
    
    X_plot_np = X_plot.numpy().flatten()
    ax.plot(X_plot_np, y_mean.flatten(), 'r-', lw=2, label='Posterior mean')
    ax.fill_between(X_plot_np, 
                    y_mean.flatten() - 2*y_std.flatten(),
                    y_mean.flatten() + 2*y_std.flatten(),
                    alpha=0.3, color='red', label='±2σ uncertainty')
    
    # True function
    y_true_plot = np.sin(X_plot_np) + 0.3 * np.cos(3*X_plot_np)
    ax.plot(X_plot_np, y_true_plot, 'k--', lw=2, label='True function')
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('ADVI Bayesian Neural Network: Predictions with Uncertainty')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-4, 4)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 ADVI BNN Results:")
    print(f"   Learned noise std: {np.exp(bnn.log_noise_std.item()):.4f} (true: 0.2)")
    
else:
    print("PyTorch not available. Skipping ADVI BNN example.")

---

## 9. Variational Inference with PyMC <a name="9-pymc"></a>

**PyMC** provides built-in VI methods that make it easy to apply variational inference to complex Bayesian models:

- **ADVI**: Automatic Differentiation VI with diagonal Gaussian
- **FullRankADVI**: ADVI with full covariance Gaussian
- **SVGD**: Stein Variational Gradient Descent

### Variational Families

| Method | Variational Family | Complexity | Quality |
|--------|-------------------|------------|---------|
| ADVI | $\mathcal{N}(\mu, \text{diag}(\sigma^2))$ | $O(D)$ | Fast, may underfit correlations |
| FullRankADVI | $\mathcal{N}(\mu, \Sigma)$ | $O(D^2)$ | Captures correlations |
| SVGD | Particle-based | $O(M^2)$ | Non-parametric, flexible |

Let's apply PyMC's VI to a financial time series model!

In [ ]:
# Generate synthetic stock returns with regime-switching volatility
np.random.seed(42)

T = 1000  # Trading days
n_regimes = 2

# True regime parameters
true_mu = np.array([0.0005, -0.0002])  # Daily returns: 12.5% vs -5% annualized
true_sigma = np.array([0.01, 0.03])    # Low vol vs high vol regime
true_transition = np.array([[0.98, 0.02],  # Stay in low vol
                           [0.05, 0.95]])   # Stay in high vol

# Generate regime sequence (Markov chain)
regimes = np.zeros(T, dtype=int)
regimes[0] = 0  # Start in low vol regime
for t in range(1, T):
    regimes[t] = np.random.choice([0, 1], p=true_transition[regimes[t-1]])

# Generate returns
returns = np.random.normal(true_mu[regimes], true_sigma[regimes])

print(f"Simulated {T} days of returns")
print(f"Time in low vol regime: {100*np.mean(regimes == 0):.1f}%")
print(f"Time in high vol regime: {100*np.mean(regimes == 1):.1f}%")

# Visualize the data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

ax = axes[0]
ax.plot(returns, 'b-', alpha=0.7, lw=0.5)
ax.fill_between(range(T), returns.min(), returns.max(), 
                where=regimes == 1, alpha=0.3, color='red', label='High vol regime')
ax.set_xlabel('Time')
ax.set_ylabel('Daily Return')
ax.set_title('Simulated Stock Returns with Regime-Switching Volatility')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
rolling_vol = pd.Series(returns).rolling(21).std() * np.sqrt(252)
ax.plot(rolling_vol, 'b-', lw=1.5, label='21-day rolling vol (annualized)')
ax.axhline(true_sigma[0] * np.sqrt(252), color='green', linestyle='--', 
           label=f'True low vol: {true_sigma[0]*np.sqrt(252):.1%}')
ax.axhline(true_sigma[1] * np.sqrt(252), color='red', linestyle='--',
           label=f'True high vol: {true_sigma[1]*np.sqrt(252):.1%}')
ax.set_xlabel('Time')
ax.set_ylabel('Annualized Volatility')
ax.set_title('Rolling Volatility')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Build a Bayesian stochastic volatility model with PyMC
# We'll use a simpler model than full HMM for VI compatibility

with pm.Model() as sv_model:
    # Priors for volatility parameters
    # Log-volatility follows AR(1) process: h_t = mu + phi*(h_{t-1} - mu) + sigma_h * eps
    
    mu_h = pm.Normal('mu_h', mu=-4, sigma=1)  # Mean log-vol (exp(-4) ≈ 0.018)
    phi = pm.Beta('phi', alpha=20, beta=1.5)   # Persistence (prior mode ~0.97)
    sigma_h = pm.Exponential('sigma_h', lam=10)  # Vol of vol
    
    # Initial log-volatility
    h_init = pm.Normal('h_init', mu=mu_h, sigma=sigma_h / np.sqrt(1 - phi**2))
    
    # Log-volatility process (simplified - using random walk approximation for VI)
    # In practice, we'd use a scan, but for VI demo we discretize
    h = pm.AR('h', rho=[phi], sigma=sigma_h, constant=True, 
              init_dist=pm.Normal.dist(mu=mu_h, sigma=0.5),
              shape=T)
    
    # Observation model
    sigma_t = pm.math.exp(h / 2)
    mu_r = pm.Normal('mu_r', mu=0, sigma=0.001)  # Mean return
    
    # Likelihood
    y = pm.Normal('y', mu=mu_r, sigma=sigma_t, observed=returns)

print("PyMC Stochastic Volatility Model defined")
print(f"Model has {sv_model.point_logps().keys()} parameters")